<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160344
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "pakistan", "city": "lahore", "street": "pakistan-lahore-cantonment", "aqicn_url": "https://api.waqi.info/feed/A74005", "latitude": 31.57, "longitude": 74.31}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [4]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
# aq_today_df = util.get_pm25(aqicn_url, country, city, street, "2024-11-05", AQI_API_KEY)
aq_today_df['date'] = pd.to_datetime(aq_today_df['date']).dt.date
aq_today_df

,pm25,country,city,street,date,url
0,198.0,pakistan,lahore,pakistan-lahore-cantonment,2024-11-16,https://api.waqi.info/feed/A74005


In [5]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   pm25     1 non-null      float32
 1   country  1 non-null      object 
 2   city     1 non-null      object 
 3   street   1 non-null      object 
 4   date     1 non-null      object 
 5   url      1 non-null      object 
dtypes: float32(1), object(5)
memory usage: 172.0+ bytes


In [6]:
from datetime import timedelta
# Generate a list of dates for the past three days (including today)
dates_list = [pd.to_datetime(today - timedelta(days=i)).tz_localize('UTC') for i in range(1,5)]  # [0, 1, 2, 3]

print("Dates to filter:", dates_list)

Dates to filter: [Timestamp('2024-11-15 00:00:00+0000', tz='UTC'), Timestamp('2024-11-14 00:00:00+0000', tz='UTC'), Timestamp('2024-11-13 00:00:00+0000', tz='UTC'), Timestamp('2024-11-12 00:00:00+0000', tz='UTC')]


In [13]:
dates_list[0]

Timestamp('2024-11-15 00:00:00+0000', tz='UTC')

In [8]:
selected_features = air_quality_fg.select(['pm25']).join(weather_fg.select_all(), on=['city'])
selected_features = selected_features.read()
# filtered_df = selected_features[selected_features['date'].isin(dates_list)]

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.41s) 


In [9]:
selected_features

,pm25,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,248.429993,2019-12-09 00:00:00+00:00,14.900082,0.2,9.422101,11.842063,lahore
1,299.160004,2019-12-10 00:00:00+00:00,14.052165,0.0,8.534353,69.443985,lahore
2,247.270004,2019-12-11 00:00:00+00:00,14.975082,2.3,8.089993,99.649574,lahore
3,53.560001,2019-12-12 00:00:00+00:00,13.347999,15.8,16.904673,72.617531,lahore
4,75.440002,2019-12-13 00:00:00+00:00,12.170915,0.0,11.525623,243.006943,lahore
...,...,...,...,...,...,...,...
1792,157.100006,2024-11-11 00:00:00+00:00,21.356337,0.5,12.081473,82.803871,lahore
1793,153.880005,2024-11-12 00:00:00+00:00,21.368834,0.0,8.260411,352.337219,lahore
1794,109.440002,2024-11-13 00:00:00+00:00,21.172998,0.0,6.034700,166.907944,lahore
1795,225.750000,2024-11-14 00:00:00+00:00,20.943834,0.0,7.204498,89.303436,lahore


In [10]:
selected_features['date'][0]

Timestamp('2019-12-09 00:00:00+0000', tz='UTC')

In [14]:
selected_features[selected_features['date'] == dates_list[1]]

,pm25,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
1795,225.750000,2024-11-14 00:00:00+00:00,20.943834,0.0,7.204498,89.303436,lahore
1796,159.800003,2024-11-14 00:00:00+00:00,20.943834,0.0,7.204498,89.303436,lahore


In [27]:
past_3_day_mean = selected_features[selected_features['date'] <= dates_list[1]][selected_features['date'] >= dates_list[3]]['pm25'].mean()

In [30]:
import numpy as np
past_3_day_mean = np.float64(past_3_day_mean)

In [32]:
aq_today_df['past_air_quality'] = past_3_day_mean

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [33]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
# daily_df['date'] = daily_df['date'].astype(str)
daily_df['city'] = city
daily_df

Coordinates 31.5°N 74.25°E
Elevation 215.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-16,19.549999,0.0,14.986553,24.102238,lahore
1,2024-11-17,21.250000,0.0,4.334974,274.763550,lahore
2,2024-11-18,21.150000,0.0,5.804825,277.124908,lahore
3,2024-11-19,20.799999,0.0,7.235910,275.710510,lahore
4,2024-11-20,20.950001,0.0,2.305125,308.659821,lahore
5,2024-11-21,21.000000,0.0,6.696387,53.746078,lahore
6,2024-11-22,22.799999,0.0,5.483356,66.801384,lahore
7,2024-11-23,22.299999,0.0,6.151683,20.556128,lahore
8,2024-11-24,22.200001,0.0,7.594207,5.440250,lahore
9,2024-11-25,20.600000,0.0,5.400000,323.130035,lahore


In [34]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [35]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-16 21:43:15,503 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160344/fs/1151047/fg/1350013


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160344/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fa42d21e9b0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 673838
         }
       },
       "result": {
         "observed_value": 198.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-16T04:43:15.000502Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "un

In [36]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-16 21:43:28,857 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160344/fs/1151047/fg/1350014


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160344/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fa42bf84460>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 673840
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-16T04:43:28.000856Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betw

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
